# Diffusionsgleichung (2D)

## Einführung

Unter der Annahme, dass die thermischen Parameter konstant sind und wir nur von einer radiogenen Wärmequelle ausgehen, ist die Wärmeleitgleichung in 2-D gegeben durch:

$$
\frac{\partial{T}}{\partial{t}} = \kappa \left( \frac{\partial^2{T}}{\partial{x^2}} + \frac{\partial^2{T}}{\partial{y}^2}\right) + \frac{Q}{\rho c_p}, \tag{1}
$$

wobei $\rho$ die Dichte in [ kg/m<sup>3</sup> ], $c_p$ die spezifische Wärmekapazität in [ J/kg/K ], $\kappa = k/\rho/c_p$ die thermische Diffusivität in [ m<sup>2</sup>/s ] und $Q$ die radiogene Wärmeproduktion pro Volumen in [ W/m<sup>3</sup> ] ist. 

Die Gleichung beschreibt die Änderung der Temperatur mit der Zeit unter der Annahme, dass die Temperatur sich nur durch Diffusion ausbreitet. 

## Das Problem

Nehmen wir and, dass die Gleichung $(1)$ in einem 2D rechteckigen Gebiet mir der Seitenlänge ($L$) 200 km und der Tiefe ($H$) 100 km definiert sei (Abb. 1). Die Temperatur an der Oberfläche sei 0 °C und steigt in dem Gebiet linear mit einem thermischen Gradienten an. Die Randbedingungen der Temperatur an den Seiten sei wählbar zwischen *Dirichlet* und *Neumann* Bedingungen. An der Unterseite des Gebietes nehmen wir in der Mitte des Rechtseckes, auf einer bestimmten Breite ($W_{Plume}$) eine höhere Temperatur als das Umgebungsgestein an. 

<img src="../Figures/Exercise05a_1.png" alt="drawing" width="450"/> <br>
**Abb. 1.** Modelaufbau 

Die beschriebene Konfiguration entspricht in etwa der Situation eines, unterhalb der Lithosphäre, stationär positioniertem Plumekopf. Die überschüssige Temperatur des Plume heizt dabei, mit der Zeit, die Lithosphäre weiter auf. 

In [45]:
using Plots, GeoModBox.HeatEquation.TwoD, ExtendableSparse

In [ ]:
FDSchema    =:explicit
# Physikalischer Parameter ---------------------------------------------- #
P       = (
    L       =   100e3,          #   Länge des Models    [m]
    H       =   200e3,          #   Höhe des Models     [m]
    k       =   6,              #   Thermische Konduktivität [W/m/K]
    cp      =   1000,           #   Wärmekapazität [J/kg/K]
    ρ       =   3200,           #   Dichte [kg/m^3]
    κ       =   [0.0],          #   Thermische Diffusivität [m^2/s]
    K0      =   273.15,         #   Kelvin bei 0 C
    Q0      =   0,              #   Hintergrund Waermeproduktionsrate;
    Tbot    =   [1300.0],       #   Temperatur am unteren Rand  [C]
    Ttop    =   [0.0],          #   Temperatur am oberen Rand   [C]
    Tplume  =   [2000.0],       #   Temperatur des Plumes [C]
    Wplume  =   50e3,           #   Breite des Plumes [m]
    Xplume  =   [0.0]
)
P.κ[1]      =   P.k / P.ρ / P.cp
P.Tbot[1]   =   P.Tbot[1]+P.K0     #   Temperatur in Kelvin
P.Ttop[1]   =   P.Ttop[1]+P.K0     #   Temperatur in Kelvin
P.Tplume[1] =   P.Tplume[1]+P.K0   #   Temperatur in Kelvin
P.Xplume[1] =   P.L[1]/2           #   X-Koordinate des Zentrums des Plumes
# ----------------------------------------------------------------------- #

In [ ]:
# Numerische Parameter -------------------------------------------------- #
NC      = (
    x       =   50,             # Gitterpunkte in x-Richtung
    y       =   50,             # Gitterpunkte in z-Richtung
)
Δ       = (
    x       = [0.0],
    y       = [0.0]

)
Δ.x[1]  =   P.L/NC.x            #   Gitterabstand in x-Richtung
Δ.y[1]  =   P.H/NC.y            #   Gitterabstand in z-Richtung
# ----------------------------------------------------------------------- #

In [ ]:
# Erstellung des Gitters ------------------------------------------------ #
x       = (
    c       =   LinRange(0.0 + Δ.x[1]/2.0, P.L - Δ.x[1]/2.0, NC.x),
)
y       = (
    c       =   LinRange(-P.H + Δ.y[1]/2.0, 0.0 - Δ.y[1]/2.0, NC.y),
)
# ----------------------------------------------------------------------- #

In [ ]:
# Zeit Parameter -------------------------------------------------------- #
T       = (
    tmax    =   [200],          #   Maximale Laufzeit des Models in Ma
    dn      =   100,             #   Inkremente der Graphischen Darstellung,
                                #   d.h. hier nur jeder 25 Zeitschritt
    day     =   3600*24,        #   Sekunden pro Tag
    year    =   [0.0],          #   Sekunden pro Jahr    
    dtfac   =   0.9,            #   Multiplikationsfaktor fuer dt
    Δ       =   [0.0],          #   Zeitschrittlaenge
    nt      =   [0]
)
T.year[1]   =   365.25*T.day            #   Sekunden pro Jahr
T.tmax[1]   =   200 * 1e6 * T.year[1]   #   Maximale Zeit in Sekunden
T.Δ[1]      =   T.dtfac*(1 / (2 * P.κ[1] *(1/Δ.x[1]^2+1/Δ.y[1]^2)))
T.nt[1]     =   floor(Int,T.tmax[1]/T.Δ[1])  # Anzahl der Zeitschritte
time        =   zeros(T.nt[1])
# ----------------------------------------------------------------------- #

In [ ]:
# Animationssettings ---------------------------------------------------- #
path        =   string("./Results/")
anim        =   Plots.Animation(path, String[] )
filename    =   string("05_2D_Plume_",FDSchema)
save_fig    =   1
# ----------------------------------------------------------------------- #

In [51]:
# Anfangstemperaturfeld ------------------------------------------------- #
D       =   (
    Q           =   zeros(NC...),
    T           =   zeros(NC...),
    T0          =   zeros(NC...),
    T_ex        =   zeros(NC.x+2,NC.y+2),    
    Tmax        =   zeros(T.nt[1]),
    Tprofile    =   zeros(NC.y,T.nt[1])
)
# Hintergrundfeld fuer Waermequellen
D.Q     .=  P.Q0
        
# Temperatur der Lithosphaere - linear zunehmenden mit der Tiefe
for i = 1:NC.x, j = 1:NC.y
    D.T[i,j]     =   P.Ttop[1] + abs(y.c[j]/P.H)*P.Tbot[1]
    if abs(x.c[i] - P.Xplume[1]) <= P.Wplume/2 && j == 1
        D.T[i,j]    =   P.Tplume[1]
    end
end
D.T0    .=  D.T
# Visualize initial condition ---
p = heatmap(x.c ./ 1e3, y.c ./ 1e3, (D.T.-P.K0)', 
        color=:viridis, colorbar=true, aspect_ratio=:equal, 
        xlabel="x [km]", ylabel="z [km]", 
        title="Temperature", 
        xlims=(0, P.L/1e3), ylims=(-P.H/1e3, 0.0), 
        clims=(0, 2000))

contour!(p,x.c./1e3,y.c/1e3,D.T'.-P.K0,levels=:10,linecolor=:black)
if save_fig == 0
    display(p)
end
# ----------------------------------------------------------------------- #

## Die Lösung

### Diskretisierung

#### Gitter und Indizierung

#### Finite Differenzen Approximation

See here (Link to README.md) in examples/HeatEquation/2D

Here, we focus on the explicit and implicit approach ...

### Randbedingungen

In [ ]:
# Randbedingungen ------------------------------------------------------- #
BC      =   (type    = (W=:Neumann, E=:Neumann, N=:Dirichlet, S=:Dirichlet),
           val     = (W=zeros(NC.y),E=zeros(NC.y),N=D.T[:,end],S=D.T[:,1]))
#BC     = (type    = (W=:Dirichlet, E=:Dirichlet, N=:Dirichlet, S=:Dirichlet),
#             val     = (W=D.T[1,:],E=D.T[end,:],N=D.T[:,end],S=D.T[:,1]))
# ----------------------------------------------------------------------- #

### Lösung des Problems

#### Gleichungssystem

In [ ]:
# Linear System of Equations -------------------------------------------- #
Num     =   (T=reshape(1:NC.x*NC.y, NC.x, NC.y),)
ndof    =   maximum(Num.T)
K       =   ExtendableSparseMatrix(ndof,ndof)
rhs     =   zeros(ndof)
# ----------------------------------------------------------------------- #

#### Zeitschleife

In [ ]:
# Zeitschleife ---------------------------------------------------------- #
for n = 1:T.nt[1]
    println(n)
    # Speicher das Temperaturprofil bei x = L/2
    @. D.Tprofile[:,n]  =  (D.T[convert(Int,NC.x/2),:] 
                                + D.T[convert(Int,NC.x/2)+1,:]) / 2
    D.Tmax[n]        =   maximum((D.T[:,convert(Int,NC.y/2)] 
                                + D.T[:,convert(Int,NC.y/2)+1]) / 2)

    if n>1
        if FDSchema==:explicit             
            ForwardEuler_const!(D, P.κ[1], Δ.x[1], Δ.y[1], T.Δ[1], P.ρ, P.cp, NC, BC)
        elseif FDSchema==:implicit
            BackwardEuler_const!(D, P.κ[1], Δ.x[1], Δ.y[1], T.Δ[1], P.ρ, P.cp, NC, BC, rhs, K, Num)
        end                            
        time[n]     =   time[n-1] + T.Δ[1]
    end
    D.T0        .=   D.T
    
    if mod(n,T.dn) == 0 || n == 1 || n == T.nt[1]
        p = heatmap(x.c ./ 1e3, y.c ./ 1e3, (D.T.-P.K0)', 
        color=:viridis, colorbar=true, aspect_ratio=:equal, 
        xlabel="x [km]", ylabel="z [km]", 
        title="Temperature", 
        xlims=(0, P.L/1e3), ylims=(-P.H/1e3, 0.0), 
        clims=(0, 2000))

        contour!(p,x.c./1e3,y.c/1e3,D.T'.-P.K0,levels=:10,linecolor=:black)
        if save_fig == 1
            Plots.frame(anim)
        else
            display(p)                        
        end
    end
end

## Results

In [ ]:
q = plot(D.Tprofile[:,1:T.dn:end].-P.K0,y.c./1e3,
        label="",xlabel="T_{x=L/2}",ylabel="Depth [km]",
        title="Temperature profile",
        layout=(1,2),subplot=1)

plot!(q,time./T.year/1e6,D.Tmax.-P.K0,
        label="",xlabel="Time [My]",ylabel="T_{max} [°C]",
        subplot=2)

if save_fig == 1
        # Write the frames to a GIF file
        Plots.gif(anim, string( path, filename, ".gif" ), fps = 15)        
        savefig(string("./Results/05_Plume_TProfile_Tmax_",FDSchema,".png"))
else
    display(q)
end
# foreach(rm, filter(endswith(".png"), readdir(path,join=true)))